In [33]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cosine


In [34]:
customers_df = pd.read_csv('Customers.csv')
transactions_df = pd.read_csv('Transactions.csv')
products_df = pd.read_csv('Products.csv')

In [35]:
transactions_products = transactions_df.merge(products_df, on='ProductID')
customer_data = transactions_products.groupby('CustomerID').agg(
    total_s=('TotalValue', 'sum'),
    avg_value=('TotalValue', 'mean'),
    total_i=('Quantity', 'sum')
).reset_index()
customer_profiles = customers_df.merge(customer_data, on='CustomerID', how='left').fillna(0)

In [36]:
features = ['total_s', 'avg_value', 'total_i']
scaler = MinMaxScaler()
customer_profiles[features] = scaler.fit_transform(customer_profiles[features])

In [37]:
similarity_matrix = []
customer_ids = customer_profiles['CustomerID'].tolist()

for i, customer1 in customer_profiles.iterrows():
    similarities = []
    for j, customer2 in customer_profiles.iterrows():
        if i != j:
            similarity = 1 - cosine(customer1[features], customer2[features])
        else:
            similarity = -1 
        similarities.append((customer2['CustomerID'], similarity))
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:3]  
    similarity_matrix.append({'CustomerID': customer1['CustomerID'], 'Similarities': similarities})

c:\Users\sanikakadam\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\Users\sanikakadam\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\Users\sanikakadam\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\Users\sanikakadam\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)
c:\Users\sanikakadam\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\spatial\distance.py:636: RuntimeWarning: invalid value encountered in scalar divide
  dist 

In [42]:
output = []
for entry in similarity_matrix:
    customer_id = entry['CustomerID']
    for similar in entry['Similarities']:
        output.append({
            'CustomerID': customer_id,
            'SimilarCustomerID': similar[0],
            'SimilarityScore': similar[1]
        })

lookalike_df = pd.DataFrame(output)
lookalike_file_path = 'output/Sanika_Kadam_Lookalike.csv'
lookalike_df.to_csv('output/Sanika_Kadam_Lookalike.csv', index=False)
lookalike_file_path if os.path.exists('output/Sanika_Kadam_Lookalike.csv') else "File creation failed."

'output/Sanika_Kadam_Lookalike.csv'